In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder


In [18]:

# Load Dataset
df = pd.read_csv("weatherAUS.csv") 
df.head()


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           145460 non-null  object 
 1   Location       145460 non-null  object 
 2   MinTemp        143975 non-null  float64
 3   MaxTemp        144199 non-null  float64
 4   Rainfall       142199 non-null  float64
 5   Evaporation    82670 non-null   float64
 6   Sunshine       75625 non-null   float64
 7   WindGustDir    135134 non-null  object 
 8   WindGustSpeed  135197 non-null  float64
 9   WindDir9am     134894 non-null  object 
 10  WindDir3pm     141232 non-null  object 
 11  WindSpeed9am   143693 non-null  float64
 12  WindSpeed3pm   142398 non-null  float64
 13  Humidity9am    142806 non-null  float64
 14  Humidity3pm    140953 non-null  float64
 15  Pressure9am    130395 non-null  float64
 16  Pressure3pm    130432 non-null  float64
 17  Cloud9am       89572 non-null

In [20]:
df.isnull().sum()

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

 ### Step-by-Step Missing Value Handling (with Explanation)
**Assumptions:**
Numerical features: median imputation

Categorical features: "Unknown" label

Columns with excessive missingness (Evaporation, Sunshine, Cloud9am, Cloud3pm) can be dropped or handled based on EDA and modeling preference

In [21]:
df.isnull().sum().sort_values(ascending=False)

Sunshine         69835
Evaporation      62790
Cloud3pm         59358
Cloud9am         55888
Pressure9am      15065
Pressure3pm      15028
WindDir9am       10566
WindGustDir      10326
WindGustSpeed    10263
Humidity3pm       4507
WindDir3pm        4228
Temp3pm           3609
RainTomorrow      3267
Rainfall          3261
RainToday         3261
WindSpeed3pm      3062
Humidity9am       2654
WindSpeed9am      1767
Temp9am           1767
MinTemp           1485
MaxTemp           1261
Date                 0
Location             0
dtype: int64

In [22]:
# Drop high-missing-value columns (optional based on project goal)
high_missing_cols = ['Evaporation', 'Sunshine', 'Cloud9am', 'Cloud3pm']
df.drop(columns=high_missing_cols, inplace=True)


In [23]:
# Identify column types
cat_cols = df.select_dtypes(include='object').columns.tolist()
num_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Exclude target from preprocessing
target_col = 'RainTomorrow'
cat_cols.remove(target_col)


In [25]:
# Impute missing values

# 1. Categorical - fill with 'Unknown'
df[cat_cols] = df[cat_cols].fillna('Unknown')

# 2. Numerical - fill with median
for col in num_cols:
    df[col] = df[col].fillna(df[col].median())


In [26]:
# Check missing values again
df.isnull().sum().sort_values(ascending=False)


RainTomorrow     3267
Location            0
Date                0
MinTemp             0
MaxTemp             0
WindGustDir         0
Rainfall            0
WindDir9am          0
WindDir3pm          0
WindSpeed9am        0
WindGustSpeed       0
WindSpeed3pm        0
Humidity9am         0
Pressure9am         0
Humidity3pm         0
Pressure3pm         0
Temp9am             0
Temp3pm             0
RainToday           0
dtype: int64

In [27]:
# Cell 6: Drop rows with target label missing
df.dropna(subset=['RainTomorrow'], inplace=True)

# Confirm clean dataset
print("✅ Final shape:", df.shape)
print("✅ Missing values:\n", df.isnull().sum().sum())


✅ Final shape: (142193, 19)
✅ Missing values:
 0


In [35]:
# Cell 2: Load dataset
data = pd.read_csv("weatherAUS.csv")

In [36]:
# Cell 3: Handle missing values
# Drop columns with more than 50% missing values
threshold = 0.5
missing_ratio = data.isnull().mean()
data = data.drop(columns=missing_ratio[missing_ratio > threshold].index)

# Fill numerical columns with median and categorical with mode
for col in data.columns:
    if data[col].dtype in ["float64", "int64"]:
        data[col] = data[col].fillna(data[col].median())
    elif data[col].dtype == "object":
        data[col] = data[col].fillna(data[col].mode()[0])

In [37]:
# Cell 4: Encode categorical features
label_encoders = {}
for col in data.select_dtypes(include=["object"]).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [38]:
# Cell 5: Define features and target
X = data.drop(columns=["RainTomorrow", "Date"])
y = data["RainTomorrow"]

In [39]:
# Cell 6: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# # Cell 7: Define Optuna objective function
# def objective(trial):
#     params = {
#         "objective": "binary",
#         "metric": "auc",
#         "boosting_type": "gbdt",
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
#         "num_leaves": trial.suggest_int("num_leaves", 20, 150),
#         "max_depth": trial.suggest_int("max_depth", 3, 15),
#         "min_child_samples": trial.suggest_int("min_child_samples", 10, 100),
#         "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),
#         "bagging_fraction": trial.suggest_float("bagging_fraction", 0.5, 1.0),
#         "scale_pos_weight": trial.suggest_float("scale_pos_weight", 1.0, 10.0)
#     }

#     train_set = lgb.Dataset(X_train, label=y_train)
#     valid_set = lgb.Dataset(X_test, label=y_test, reference=train_set)
# # 
#     model = lgb.train(
#         params,
#         train_set,
#         valid_sets=[valid_set],
#         num_boost_round=200,
#         early_stopping_rounds=20,
#         verbose_eval=False
#     )

#     preds = model.predict(X_test)
#     return roc_auc_score(y_test, preds)


In [55]:
def objective(trial):
    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.2),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1.0, 10.0),
        'objective': 'binary',
        'boosting_type': 'gbdt',
        'verbosity': -1,
        'n_jobs': -1
    }

    model = lgb.LGBMClassifier(**params)

    from lightgbm import early_stopping

    model.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric='auc',
        callbacks=[early_stopping(stopping_rounds=20)],
    )

    preds = model.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, preds)


In [56]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)


[I 2025-06-28 14:32:59,871] A new study created in memory with name: no-name-e2e91e7b-3520-464e-ba81-fd480fffa733


Training until validation scores don't improve for 20 rounds


[I 2025-06-28 14:33:02,997] Trial 0 finished with value: 0.863422556429823 and parameters: {'learning_rate': 0.10707190082652261, 'num_leaves': 146, 'max_depth': 8, 'min_child_samples': 10, 'feature_fraction': 0.8169442622301717, 'bagging_fraction': 0.6381726025586281, 'scale_pos_weight': 7.361953218356355}. Best is trial 0 with value: 0.863422556429823.


Early stopping, best iteration is:
[5]	valid_0's auc: 0.863423	valid_0's binary_logloss: 0.472047
Training until validation scores don't improve for 20 rounds


[I 2025-06-28 14:33:06,289] Trial 1 finished with value: 0.8699151841952526 and parameters: {'learning_rate': 0.1085019167073011, 'num_leaves': 121, 'max_depth': 9, 'min_child_samples': 87, 'feature_fraction': 0.9652996540121415, 'bagging_fraction': 0.6337189414916309, 'scale_pos_weight': 3.9969872877992585}. Best is trial 1 with value: 0.8699151841952526.


...

Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.891187	valid_0's binary_logloss: 0.331528
Training until validation scores don't improve for 20 rounds


[I 2025-06-28 17:44:53,335] Trial 48 finished with value: 0.8591583694161021 and parameters: {'learning_rate': 0.09844572554674205, 'num_leaves': 128, 'max_depth': 13, 'min_child_samples': 96, 'feature_fraction': 0.7361668576145993, 'bagging_fraction': 0.5689001920103346, 'scale_pos_weight': 8.025514999001384}. Best is trial 27 with value: 0.8912415845804287.


Early stopping, best iteration is:
[4]	valid_0's auc: 0.859158	valid_0's binary_logloss: 0.478304
Training until validation scores don't improve for 20 rounds


[I 2025-06-28 17:44:54,943] Trial 49 finished with value: 0.8764788713815551 and parameters: {'learning_rate': 0.11416645365719874, 'num_leaves': 118, 'max_depth': 4, 'min_child_samples': 88, 'feature_fraction': 0.768529480890116, 'bagging_fraction': 0.7050181331859798, 'scale_pos_weight': 1.429272346451622}. Best is trial 27 with value: 0.8912415845804287.


Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.876479	valid_0's binary_logloss: 0.350377


In [59]:
# Assuming `study` is your completed Optuna study
best_trial = study.best_trial

print("Best Trial:")
print(f"  Value (AUC): {best_trial.value}")
print("  Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")


Best Trial:
  Value (AUC): 0.8912415845804287
  Best Hyperparameters:
    learning_rate: 0.12135248320669911
    num_leaves: 114
    max_depth: 11
    min_child_samples: 100
    feature_fraction: 0.5512734781909172
    bagging_fraction: 0.5929453470563474
    scale_pos_weight: 2.19693154497164


In [60]:
# If you want to save these best parameters as a dictionary for later use:
best_params = best_trial.params
best_params['objective'] = 'binary'
best_params['boosting_type'] = 'gbdt'
best_params['verbosity'] = -1
best_params['n_jobs'] = -1


In [61]:
# You can then use best_params to retrain your model like this:
final_model = lgb.LGBMClassifier(**best_params)
final_model.fit(X_train, y_train)


,boosting_type,'gbdt'
,num_leaves,114
,max_depth,11
,learning_rate,0.12135248320669911
,n_estimators,100
,subsample_for_bin,200000
,objective,'binary'
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,100


In [62]:
# Predict probability of rain
y_pred_proba = final_model.predict_proba(X_test)[:, 1]

# Convert probabilities to binary prediction (0 = No Rain, 1 = Rain)
y_pred = (y_pred_proba >= 0.5).astype(int)


In [63]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

print("AUC Score:", roc_auc_score(y_test, y_pred_proba))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


AUC Score: 0.8912415845804287
Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.89      0.90     22672
           1       0.63      0.69      0.66      6420

    accuracy                           0.84     29092
   macro avg       0.77      0.79      0.78     29092
weighted avg       0.85      0.84      0.85     29092

Confusion Matrix:
 [[20066  2606]
 [ 1975  4445]]


### (Optional) Save the Model
If you want to use the model later:

```python
import joblib
joblib.dump(final_model, 'rain_prediction_model.pkl')
```


### (Optional) Predict on New Data
To predict rainfall for tomorrow’s conditions:

```python
# Example new sample (make sure it’s preprocessed the same way as your training set)
new_sample = pd.DataFrame([{
    'MinTemp': 10.2,
    'MaxTemp': 21.5,
    'Rainfall': 0.0,
    'Humidity9am': 80,
    'Humidity3pm': 65,
    # Add all other required features...
}])

# Apply same preprocessing as training data
# ...

# Predict rain
rain_prob = final_model.predict_proba(new_sample)[:, 1]
rain_prediction = (rain_prob >= 0.5).astype(int)

print("Will it rain tomorrow?", "Yes" if rain_prediction[0] == 1 else "No")

```

In [66]:
import joblib
joblib.dump(final_model, 'rain_prediction_model.pkl')

['rain_prediction_model.pkl']

In [67]:
print(X_train.columns.tolist())


['Location', 'MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine', 'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm', 'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm', 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainToday']
